# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.121592e+02     1.396676e+00
 * time: 0.6430199146270752
     1     1.090356e+01     8.257016e-01
 * time: 1.9149219989776611
     2    -1.190047e+01     9.160670e-01
 * time: 2.489764928817749
     3    -3.422187e+01     7.386412e-01
 * time: 3.391774892807007
     4    -4.786018e+01     5.638092e-01
 * time: 4.250895977020264
     5    -5.703428e+01     2.021741e-01
 * time: 5.222264051437378
     6    -5.991274e+01     1.251394e-01
 * time: 5.796416997909546
     7    -6.103811e+01     4.901186e-02
 * time: 6.36556601524353
     8    -6.146000e+01     6.187852e-02
 * time: 6.969156980514526
     9    -6.173018e+01     4.136818e-02
 * time: 7.574378967285156
    10    -6.192446e+01     2.209907e-02
 * time: 8.15067195892334
    11    -6.206388e+01     1.735738e-02
 * time: 8.730110883712769
    12    -6.211764e+01     1.368694e-02
 * time: 9.31196904182434
    13    -6.216884e+01     1.211800e-02
 * time: 9.933525085449219
    14  

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671064
    AtomicLocal         -18.8557668
    AtomicNonlocal      14.8522643
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485365 
    Xc                  -19.3336817

    total               -62.261666460172
